# 既成モデルを利用して顔認証を最速で実装する
- `MTCNN`, `keras_vggface`を利用した顔認証の実装例です。
- 2019年時点では本当に早かったのですが今(Mar.11.2020)動かすと`keras`のAPI変更などのため微妙です。

### 1. 画像ロード
- 何でもよいので画像を読み込みます。
  - この例では`True`を渡すことで、Webカメラなりのソースを`VideoCapture`するようにしました。
  - `False`を渡すといつもの人を読み込みます。

In [22]:
# 基本セットのimport
import cv2
import numpy as np
import matplotlib.pyplot as plt

def get_image(useCam):
    if useCam:
        cap = cv2.VideoCapture(0)
        _,origin = cap.read()
        cap.release()
        origin = cv2.cvtColor(origin, cv2.COLOR_BGR2RGB)
    else:
        origin = plt.imread('https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png')
    return origin

### 2. 顔の検出
- 極論やらなくてもよいのですが、顔の部分だけ抜粋してあげた方が親切だよねということで、顔の検出を行います。
- 一枚の画像に複数の顔があり、それぞれ認証したいです、というような場合はここで工夫してあげる必要があります。
- ※2020年現在`MTCNN`が`keras`の最新APIにキャッチアップしていません。
  - https://github.com/ipazc/mtcnn/issues/83
  - 上記issueのとおり`factory.py`を編集するとエラー自体は回避できます。
  - 抽出もうまくいかないようですが、メンテナンスは継続しているようなので、そのうち対応してくれるかもしれません。
- 今回は、顔抽出に失敗した場合は画像をそのまま特徴抽出に流すものとして進めます。

In [40]:
import mtcnn
import PIL.Image as Image
detector = mtcnn.MTCNN()

# 顔抽出メソッド
def extract_face(source):
    face = source
    faces = detector.detect_faces(source)
    print(f'抽出した顔の数:{len(faces)}')
    if len(faces) > 0:
        # 開始ピクセルのX,Yおよび判定結果の横幅、縦幅を取得
        x1, y1, w, h, = faces[0]['box']
        # 終端ピクセルを決定
        x2, y2 = x1 + w, y1 + h
        face = source[y1:y2, x1:x2]
        image = Image.fromarray(face)
    else:
        image = Image.fromarray((face * 255).astype(np.uint8))
    resized = image.resize((224, 224), Image.ANTIALIAS)
    return resized

In [41]:
# モデルに渡すための加工メソッド
import numpy

def to_processable(face):
    pixels = numpy.asarray(face)
    pixels = pixels.astype('float32')
    return numpy.expand_dims(pixels, axis=0)

### 3.特徴の抽出
- `vggface`が提供するモデルに画像を渡して、特徴を取得します。
  - 試した限り、英語圏の人物にはかなり優秀ですが、日本人についてはやや信頼性が落ちるようです。
    - 実際問題それはやむを得ないところなので、精度を出したい場合は日本人の顔データで転移学習なり
- ※`keras_vggface`においても`MTCNN`同様、`keras`から`tensorflow.keras`への移行が必要となります。

In [42]:
import keras_vggface as vgg
import keras_vggface.utils as vggutil

# ここまで作ったメソッドを呼び出し、顔データを取得
source = get_image(False)
face = extract_face(source)
samples = to_processable(face)

# モデルに顔データを渡す
samples = vggutil.preprocess_input(samples, version=2)
model = vgg.VGGFace(model='resnet50', include_top=False, input_shape=(224,224,3), pooling='avg')
features = model.predict(samples)

抽出した顔の数:0


### 4. 特徴の比較
- 特徴(ベクター)間の距離を比較することで、同一人物であるか別人であるかの判定を行います。
- 今回のセットアップでは`100`程度を閾値としてあげると丁度よい感じになります。

In [44]:
# 二枚目の画像を用意し、特徴を抽出
face2 = get_image(False)
face2 = extract_face(face2)
samples2 = to_processable(face2)
feature2 = model.predict(samples2)

抽出した顔の数:0


In [46]:
# 比較
import scipy.spatial
dist = scipy.spatial.distance.euclidean(features, feature2)
if (dist < 100):
    print(f'同一人物です (特徴差スコア：{dist})')
else:
    print(f'違う人物です (特徴差スコア：{dist})')

同一人物です (特徴差スコア：19.01454734802246)
